In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string
import tensorflow as tf
#import random
import numpy as np
import random
from ast import literal_eval
from datasets import Dataset
from transformers import AutoTokenizer


2024-05-10 11:26:26.802236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-10 11:26:26.802319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-10 11:26:26.943548: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def custom_standardization(input_data):
#     lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(input_data,'[%s]' % re.escape(string.punctuation), '').numpy().decode('utf-8')

def remove_alphanumeric(text):
    return re.sub('\w*\d\w*', '', text)
def remove_extra_spaces(text):
    cleaned_text =' '.join(text.split())
    return ''.join(char for char in cleaned_text if (not char.isdigit()) and (char in string.printable))

In [3]:
tsv_file_path = "your_file.tsv"
columns=["unsplitted", "splitted"]
wikiDF = pd.read_csv('/kaggle/input/wikisplit/wiki-split-master/train.tsv/train.tsv', sep='\t',names=columns)
wikiDF=wikiDF.head(10000)

In [4]:
wikiDF['cleaned_title']=wikiDF['unsplitted'].apply(custom_standardization)
wikiDF['cleaned_title']=wikiDF['cleaned_title'].apply(remove_alphanumeric)
wikiDF['cleaned_title']=wikiDF['cleaned_title'].apply(remove_extra_spaces)

In [5]:
wikiDF.head()
wikiDF.iloc[0,2]

'As she translates from one language to another she tries to find the appropriate wording and context in English that would correspond to the work in Spanish her poems and stories started to have differing meanings in their respective languages'

In [6]:
cleanedwikiDF=wikiDF.drop(columns=columns)
#save the cleaned data
cleanedwikiDF.to_csv('cleaned_wiki.csv',index=False)
loadedwikiDF=pd.read_csv('/kaggle/working/cleaned_wiki.csv')
loadedwikiDF.head()

,cleaned_title
0,As she translates from one language to another...
1,BDSM is solely based on consensual activities ...
2,Critics criticized the use of the disputed fig...
3,Do Re Mi is a song by Kurt Cobain of the band ...
4,For Robert Price docetism together with encrat...


In [7]:
def convertTolist(loadedwikiDF):
    loadedwikiDF['cleaned_title_list']= loadedwikiDF['cleaned_title'].apply(lambda x: x.split(' '))
    #remove whitespaces and empty elements
    loadedwikiDF['cleaned_title_list']=loadedwikiDF['cleaned_title_list'].apply(lambda x: list(filter(None, x)))
    return loadedwikiDF
convertTolist(loadedwikiDF)

,cleaned_title,cleaned_title_list
0,As she translates from one language to another...,"[As, she, translates, from, one, language, to,..."
1,BDSM is solely based on consensual activities ...,"[BDSM, is, solely, based, on, consensual, acti..."
2,Critics criticized the use of the disputed fig...,"[Critics, criticized, the, use, of, the, dispu..."
3,Do Re Mi is a song by Kurt Cobain of the band ...,"[Do, Re, Mi, is, a, song, by, Kurt, Cobain, of..."
4,For Robert Price docetism together with encrat...,"[For, Robert, Price, docetism, together, with,..."
...,...,...
9995,A MacGuffin is an object or character which dr...,"[A, MacGuffin, is, an, object, or, character, ..."
9996,A Madea Halloween and Keeping Up with the Jone...,"[A, Madea, Halloween, and, Keeping, Up, with, ..."
9997,A Madea Halloween and starred in the YouTube R...,"[A, Madea, Halloween, and, starred, in, the, Y..."
9998,A MagSafe cable runs from the back of the disp...,"[A, MagSafe, cable, runs, from, the, back, of,..."


In [8]:
loadedwikiDF

,cleaned_title,cleaned_title_list
0,As she translates from one language to another...,"[As, she, translates, from, one, language, to,..."
1,BDSM is solely based on consensual activities ...,"[BDSM, is, solely, based, on, consensual, acti..."
2,Critics criticized the use of the disputed fig...,"[Critics, criticized, the, use, of, the, dispu..."
3,Do Re Mi is a song by Kurt Cobain of the band ...,"[Do, Re, Mi, is, a, song, by, Kurt, Cobain, of..."
4,For Robert Price docetism together with encrat...,"[For, Robert, Price, docetism, together, with,..."
...,...,...
9995,A MacGuffin is an object or character which dr...,"[A, MacGuffin, is, an, object, or, character, ..."
9996,A Madea Halloween and Keeping Up with the Jone...,"[A, Madea, Halloween, and, Keeping, Up, with, ..."
9997,A Madea Halloween and starred in the YouTube R...,"[A, Madea, Halloween, and, starred, in, the, Y..."
9998,A MagSafe cable runs from the back of the disp...,"[A, MagSafe, cable, runs, from, the, back, of,..."


In [9]:
def replacementOfCharacters(text):
    Characterlen=len(text)
    randomIndex=random.randint(0,Characterlen-1)
    #to lowercase
    randomCharacter=random.choice(string.ascii_letters[:26])
    text=text[:randomIndex]+randomCharacter+text[randomIndex+1:]
    return text
replacementOfCharacters('hello')

'hellp'

In [10]:
def SwappingCharacters(text):
    Characterlen=len(text)
    while(True):
        randomIndex=random.randint(0,Characterlen-1)
        randomIndex2=random.randint(randomIndex-1,randomIndex+1)

        if(randomIndex!=randomIndex2 and randomIndex2<=Characterlen-1 and randomIndex2>=0):
            swap1=text[randomIndex]
            swap2=text[randomIndex2]
            if(swap1!=swap2):
                break

    #swap the text in the random index with the random index2
    # print(randomIndex,randomIndex2)
    textlist=list(text)
    textlist[randomIndex]=swap2
    textlist[randomIndex2]=swap1
    text=''.join(textlist)
    return text
    # return text
SwappingCharacters('hell0world')

'hellw0orld'

In [11]:
def deleteRandomChar(text):
    Characterlen=len(text)
    randomIndex=random.randint(0,Characterlen-1)
    text=text[:randomIndex]+text[randomIndex+1:]
    return text
deleteRandomChar('hello')

'hell'

In [12]:
def insertRandomChar(text):
    Characterlen=len(text)
    randomIndex=random.randint(0,Characterlen)
    randomCharacter=random.choice(string.ascii_letters[:26])
    text=text[:randomIndex]+randomCharacter+text[randomIndex:]
    return text
insertRandomChar('hello')

'nhello'

In [13]:
# missedupsplitted

NameError: name 'missedupsplitted' is not defined

In [14]:
missedupsplitted=loadedwikiDF.copy()
def truncateList(textlist, N):
    if len(textlist)>N:
        textlist=textlist[:N]
    return textlist
missedupsplitted['cleaned_title_list']=missedupsplitted['cleaned_title_list'].apply(lambda x: truncateList(x,15))

In [15]:
def missupDataAtrandom(listOfwordss):
    np.random.seed(42)
    listOfwords=listOfwordss.copy()
    numberOfmissingups=np.random.randint(0,min(4,len(listOfwords)))
#     numberOfmissingups=0
#     print(numberOfmissingups)
#     print(listOfwords)
    for i in range(numberOfmissingups+1):
#         print(i)
        typeOfMiss=np.random.randint(0,4)
#         print("type: ",typeOfMiss)
        indexTomiss=np.random.randint(0,len(listOfwordss))
#         print("index: ",indexTomiss)
        while len(listOfwords[indexTomiss])<=1:
            indexTomiss=np.random.randint(0,len(listOfwords))
        if len(listOfwords[indexTomiss])>1:
            if typeOfMiss==0:
                listOfwords[indexTomiss]= replacementOfCharacters(listOfwords[indexTomiss])
            elif typeOfMiss==1:
                continue
                listOfwords[indexTomiss]= SwappingCharacters(listOfwords[indexTomiss])
            elif typeOfMiss==2:
                listOfwords[indexTomiss]= deleteRandomChar(listOfwords[indexTomiss])
            else:
                listOfwords[indexTomiss]= insertRandomChar(listOfwords[indexTomiss])
    return listOfwords
    # return randomint
missupDataAtrandom(['hello','world','how','are','you'])


['hello', 'world', 'hw', 'are', 'vydou']

In [16]:


missedupsplitted['missedup_abstract_list']=missedupsplitted['cleaned_title_list'].apply(missupDataAtrandom)
missedupsplitted.head()


,cleaned_title,cleaned_title_list,missedup_abstract_list
0,As she translates from one language to another...,"[As, she, translates, from, one, language, to,...","[As, she, translates, from, one, language, to,..."
1,BDSM is solely based on consensual activities ...,"[BDSM, is, solely, based, on, consensual, acti...","[BDSM, is, solely, based, on, consensual, acti..."
2,Critics criticized the use of the disputed fig...,"[Critics, criticized, the, use, of, the, dispu...","[Critics, criticized, the, use, of, the, dispu..."
3,Do Re Mi is a song by Kurt Cobain of the band ...,"[Do, Re, Mi, is, a, song, by, Kurt, Cobain, of...","[Do, Re, Mi, is, a, song, by, Kurt, Cobain, of..."
4,For Robert Price docetism together with encrat...,"[For, Robert, Price, docetism, together, with,...","[For, Robert, Price, docetism, together, with,..."


In [17]:
missedupsplitted.iloc[0,1]==missedupsplitted.iloc[0,2]
print(missedupsplitted.iloc[0,1])
print(missedupsplitted.iloc[0,2])

['As', 'she', 'translates', 'from', 'one', 'language', 'to', 'another', 'she', 'tries', 'to', 'find', 'the', 'appropriate', 'wording']
['As', 'she', 'translates', 'from', 'one', 'language', 'to', 'another', 'she', 'tries', 't', 'find', 'thkeh', 'appropriate', 'wording']


In [18]:
def convertToString(listOfWords):
    return ' '.join(listOfWords)
convertToString(['hello','world','how','are','you'])
missedupsplitted.drop(columns=['cleaned_title'],inplace=True)

missedupsplitted['missedup_abstract']=missedupsplitted['missedup_abstract_list'].apply(convertToString)
missedupsplitted['cleaned_title']=missedupsplitted['cleaned_title_list'].apply(convertToString)
missedupsplitted.drop(columns=['missedup_abstract_list','cleaned_title_list'],inplace=True)

In [19]:
missedupsplitted.head()

,missedup_abstract,cleaned_title
0,As she translates from one language to another...,As she translates from one language to another...
1,BDSM is solely based on consensual activities ...,BDSM is solely based on consensual activities ...
2,Critics criticized the use of the disputed fig...,Critics criticized the use of the disputed fig...
3,Do Re Mi is a song by Kurt Cobain of te band N...,Do Re Mi is a song by Kurt Cobain of the band ...
4,For Robert Price docetism together with encrat...,For Robert Price docetism together with encrat...


In [20]:
# from datasets import Dataset
# hfDF=Dataset.from_pandas(missedupsplitted,split='train')
hfDF=Dataset.from_pandas(missedupsplitted)
hfDF

Dataset({
    features: ['missedup_abstract', 'cleaned_title'],
    num_rows: 10000
})

In [21]:
# hfDF['cleaned_title']

In [22]:
# def getmaxlen(huggingfaceDf):
#     maxLen=0
#     for i in range(len(huggingfaceDf)):
# #         print(len(huggingfaceDf['cleaned_title'][i]))
#         maxLen=max(maxLen,len(huggingfaceDf['cleaned_title'][i]))
#     return maxLen
# getmaxlen(hfDF)

In [23]:
# as tarket tokenizer explaination
# This is because the languages you're translating between (Engish and German in this case) have 
# different tokenization vocabularies. This implies that tokens will get tokenized 
# differently. MarianMT models have seq2seq (encoder-decoder) architectures, and both the encoder and 
# decoder each have their own embedding matrix. This means that the encoder will have an embedding vector 
# for the token '▁doctor', whereas the decoder will learn an embedding vector for the token '▁do', an
# embedding vector for the token 'ctor', etc.

# Tokenization vocabularies are typically built per language (although models like BLOOM just have one 
# large vocabulary for all language tokens).

In [24]:

model_checkpoint='facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [25]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [0, 31414, 6, 42, 16, 10, 3645, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [26]:
max_input_length=100
max_target_length=100
def transProcessing(data):
    inputs=data['missedup_abstract']
    
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(data['cleaned_title'], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
modelinputs=transProcessing(hfDF[:2])
modelinputs

{'input_ids': [[0, 1620, 79, 19303, 31, 65, 2777, 7, 277, 79, 5741, 326, 465, 3553, 1071, 298, 3901, 28776, 2], [0, 387, 5433, 448, 16, 9382, 716, 15, 26798, 1713, 8, 716, 15, 16, 467, 17377, 463, 2074, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[0, 1620, 79, 19303, 31, 65, 2777, 7, 277, 79, 5741, 7, 465, 5, 3901, 28776, 2], [0, 387, 5433, 448, 16, 9382, 716, 15, 26798, 1713, 8, 716, 15, 63, 467, 8, 2074, 5, 2]]}

In [28]:
tokenizedDS=hfDF.map(transProcessing,batched=True)
tokenizedDS


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['missedup_abstract', 'cleaned_title', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [29]:
from transformers import TFAutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,TFTrainingArguments
model=TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
config=model.config

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [30]:
# config

In [31]:
batch_size=24
learning_rate=5e-5
weight_decay=0.01
adam_epsilon=1e-8
num_train_epochs=4

In [32]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

In [33]:
train_dataset = model.prepare_tf_dataset(
    tokenizedDS,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [34]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer,experimental_run_tf_function=False)

In [35]:
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

In [ ]:

# tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./summarization_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )

# metric_callback = KerasMetricCallback(
#     metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
# )

# callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(train_dataset, validation_data=train_dataset, epochs=num_train_epochs)

In [42]:
sentence='As she translates from one lanuage to anoher'
inputs = tokenizer(sentence, max_length=512, return_tensors='tf', padding="max_length", truncation=True)
predicted_abstract = model.generate(**inputs)
decoded=tokenizer.decode(predicted_abstract[0], skip_special_tokens=True)
decoded

'As she translates from one language to another'

metrices

In [ ]:
# print(missedupsplitted.iloc[0,2]==missedupsplitted.iloc[0,1])
https://debuggercafe.com/getting-started-with-grammar-correction/
https://learnopencv.com/fine-tuning-t5/#aioseo-the-t5-text-to-text-transformer-model

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base", forced_bos_token_id=0)
tok = BartTokenizer.from_pretrained("facebook/bart-base")
example_english_phrase = "UN Chief <mask> There Is No <mask> in <mask>"
batch = tok(example_english_phrase, return_tensors="pt")
generated_ids = model.generate(batch["input_ids"])
print(generated_ids)
# convert back the IDs
decoded=tok.decode(generated_ids[0], skip_special_tokens=True)
print(decoded)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW

# Step 1: Prepare the data
class SpellingCorrectionDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
# Step 2: Tokenize the data
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Step 3: Define the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# Modify the model's architecture for spelling correction

# Step 4: Fine-tune the model
def train(model, train_dataloader, val_dataloader, optimizer, loss_fn, num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            
            loss.backward()
            optimizer.step()
        
        avg_loss = total_loss / len(train_dataloader)
        
        # Evaluate on validation data
        model.eval()
        # Evaluation code
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')
    
# Step 5: Test the model
def correct_spelling(model, text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    outputs = model.generate(input_ids)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return corrected_text

# Example usage
train_dataset = SpellingCorrectionDataset(train_data)
val_dataset = SpellingCorrectionDataset(val_data)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train(model, train_dataloader, val_dataloader, optimizer, loss_fn, num_epochs=10)

text = "Ths is an exmple of misspelled wrds."
corrected_text = correct_spelling(model, text)
print(corrected_text)


In [ ]:
# spell checker
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Bhuvana/t5-base-spellchecker")

model = AutoModelForSeq2SeqLM.from_pretrained("Bhuvana/t5-base-spellchecker")
def correct(inputs):
    input_ids = tokenizer.encode(inputs,return_tensors='pt')
    sample_output = model.generate(
        input_ids,
        do_sample=True,
        max_length=50,
        top_p=0.99,
        num_return_sequences=1
    )
    res = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    return res


text = "christmas is celbrated on decembr 25 evry ear"

print(correct(text))